In [2]:
import os
import json
from PIL import Image
from orient_anything import get_3angle, render_3D_axis, overlay_images_with_scaling
from transformers import AutoImageProcessor
from rewards.orient import OrientLoss
from torchvision.transforms import ToTensor, ToPILImage
import torch
import numpy as np
device = torch.device("cuda:1")

orientation = [0, 90, 90]
reward_loss = OrientLoss(1.0, torch.float16, device, '/root/.cache/huggingface/hub', False)
results = {}
val_preprocess = AutoImageProcessor.from_pretrained("facebook/dinov2-large", cache_dir='./')

for img_path in sorted([path for path in os.listdir("./results/orient") if path.startswith("a photo of a car_(90,90,90)")]):
    image = Image.open(os.path.join("./results/orient", img_path, "init.png"))
    angles = get_3angle(image, reward_loss.orient_estimator, val_preprocess, device)
    key = img_path.split("_")[-1].zfill(2)  # JSON의 key 값으로 사용
    results[key] = angles[0].item()  # value 값으로 angles[0] 저장

# JSON 파일 저장
with open("./results/orient/azimuth_per_seed.json", "w") as f:
    json.dump(results, f, indent=4, sort_keys=True)


large


In [7]:
azimuths = results.values()
distribution = [[], [], [], []]
for angle in azimuths:
    if angle >= 0 and angle < 90:
        distribution[0].append(angle)
    elif angle >= 90 and angle < 180:
        distribution[1].append(angle)
    elif angle >= 180 and angle < 270:
        distribution[2].append(angle)
    elif angle >= 270 and angle < 360:
        distribution[3].append(angle)

print(len(distribution[0]))
print(len(distribution[1]))
print(len(distribution[2]))
print(len(distribution[3]))

75
4
0
21


In [9]:
print(distribution[1])

[93.0, 91.0, 161.0, 91.0]
